In [5]:
import torch
from torchvision import models
from transformers import BertModel
from fvcore.nn import FlopCountAnalysis
import numpy as np

In [6]:
model_list = [
    ("ResNet101", models.resnet101(weights=models.ResNet101_Weights.DEFAULT)),
    ("EfficientNet_B0", models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)),
    ("AlexNet", models.alexnet(weights=models.AlexNet_Weights.DEFAULT)),
    ("SqueezeNet1_0", models.squeezenet1_0(weights=models.SqueezeNet1_0_Weights.DEFAULT)),
    ("BERT_Large_Uncased", BertModel.from_pretrained('bert-large-uncased'))
]

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to C:\Users\mruba/.cache\torch\hub\checkpoints\resnet101-cd907fc2.pth
100%|███████████████████████████████████████████████████████████████████████████████| 171M/171M [00:27<00:00, 6.51MB/s]
C:\Users\mruba\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

C:\Users\mruba\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mruba\.cache\huggingface\hub\models--bert-large-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [14]:
def analyze_model(model_name, model, input_tensor):
    model.eval()
    flops = FlopCountAnalysis(model, input_tensor)
    model = model.to('cpu')
    output = model(input_tensor)

    flop_breakdown = flops.by_operator()

    max_flop = max(flop_breakdown.values())

    print(f"\nModel: {model_name}")
    print(f"Total FLOPs: {flops.total()}")
    print("\nOperator Name\t\tFLOP Count\t\tRelative %")
    print("-------------------------------------------------------------")
    for op_name, count in flop_breakdown.items():
        relative_percentage = (count / max_flop) * 100
        print(f"{op_name:<20}\t{count:<15}\t{relative_percentage:.2f}%")
    print("-------------------------------------------------------------")

In [15]:
# For ResNet101
for model_name, model in model_list:
    if model_name == "ResNet101":
        input_tensor = torch.randn(1, 3, 224, 224)
        analyze_model(model_name, model, input_tensor)

Unsupported operator aten::max_pool2d encountered 1 time(s)
Unsupported operator aten::add_ encountered 33 time(s)



Model: ResNet101
Total FLOPs: 7833969664

Operator Name		FLOP Count		Relative %
-------------------------------------------------------------
conv                	7799357440     	100.00%
batch_norm          	32463872       	0.42%
adaptive_avg_pool2d 	100352         	0.00%
linear              	2048000        	0.03%
-------------------------------------------------------------


In [17]:
for model_name, model in model_list:
    if model_name == "EfficientNet_B0":
        input_tensor = torch.randn(1, 3, 224, 224)
        analyze_model(model_name, model, input_tensor)

Unsupported operator aten::silu_ encountered 49 time(s)
Unsupported operator aten::sigmoid encountered 16 time(s)
Unsupported operator aten::mul encountered 16 time(s)
Unsupported operator aten::add_ encountered 9 time(s)
Unsupported operator aten::dropout_ encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
features.1.0.stochastic_depth, features.2.0.stochastic_depth, features.2.1.stochastic_depth, features.3.0.stochastic_depth, features.3.1.stochastic_depth, features.4.0.stochastic_depth, features.4.1.stochastic_depth, features.4.2.stochastic_depth, features.5.0.stochastic_depth, features.5.1.stochastic_depth, features.5.2.stochastic_depth, features.6.0.stochastic_depth, features.6.1.stochastic_de


Model: EfficientNet_B0
Total FLOPs: 401659392

Operator Name		FLOP Count		Relative %
-------------------------------------------------------------
conv                	384534752      	100.00%
batch_norm          	13475392       	3.50%
adaptive_avg_pool2d 	2369248        	0.62%
linear              	1280000        	0.33%
-------------------------------------------------------------


In [18]:
for model_name, model in model_list:
    if model_name == "AlexNet":
        input_tensor = torch.randn(1, 3, 224, 224)
        analyze_model(model_name, model, input_tensor)

Unsupported operator aten::max_pool2d encountered 3 time(s)



Model: AlexNet
Total FLOPs: 714197696

Operator Name		FLOP Count		Relative %
-------------------------------------------------------------
conv                	655566528      	100.00%
adaptive_avg_pool2d 	9216           	0.00%
linear              	58621952       	8.94%
-------------------------------------------------------------


In [19]:
for model_name, model in model_list:
    if model_name == "SqueezeNet1_0":
        input_tensor = torch.randn(1, 3, 224, 224)
        analyze_model(model_name, model, input_tensor)

Unsupported operator aten::max_pool2d encountered 3 time(s)



Model: SqueezeNet1_0
Total FLOPs: 819093576

Operator Name		FLOP Count		Relative %
-------------------------------------------------------------
conv                	818924576      	100.00%
adaptive_avg_pool2d 	169000         	0.02%
-------------------------------------------------------------


In [20]:
for model_name, model in model_list:
    if model_name.startswith("BERT"):
        input_tensor = torch.randint(0, 30522, (1, 512))
        analyze_model(model_name, model, input_tensor)

Unsupported operator aten::add encountered 75 time(s)
Unsupported operator aten::rsub encountered 1 time(s)
Unsupported operator aten::mul encountered 1 time(s)
Unsupported operator aten::embedding encountered 3 time(s)
Unsupported operator aten::add_ encountered 1 time(s)
Unsupported operator aten::div encountered 24 time(s)
Unsupported operator aten::softmax encountered 24 time(s)
Unsupported operator aten::gelu encountered 24 time(s)
Unsupported operator aten::tanh encountered 1 time(s)



Model: BERT_Large_Uncased
Total FLOPs: 167633223680

Operator Name		FLOP Count		Relative %
-------------------------------------------------------------
layer_norm          	128450560      	0.08%
linear              	154619871232   	100.00%
matmul              	12884901888    	8.33%
-------------------------------------------------------------
